# QUBO Formulation for Quantum Credit Scoring

In previous project reports, specifically for WP5, we defined the QUBO formulation for the rating scale definition problem. The project's aim is to divide $n$ counterparts into $m$ grades according to several constraints.

In this notebook, we present the toy model development of the cost function. Furthermore, we solve the problem using different methods and check if the achieved solutions satisfy all the requested constraints.
Each part of the code is properly commented to provide a better understanding of its functionalities.

## Libraries and hyperparameters definition

In this section, we import all the required Python libraries for the execution of the code. Additionally we set the hyperparameters of the problem.

It should be noted that we have implemented the possibility to use random data or to import it from the database provided by Intesa Sanpaolo (which is not included in this repository).

Users can also select only certain constraints in both the definition of the QUBO problem and the verification of its solution.

In [19]:
import math
import time
import itertools
import dimod
import hybrid

In [20]:
config = {
    
    'random_data': 'yes',       # select 'yes' to generate random dataset
    'data_path': 'data/dataset-ispq.csv', # could be omitted to generate random data
    
    'n_counterpart': 4,         # could be a number or 'all'
    'grades': 2,
    'default_prob': 0.4,        # probability of default (range: 0 - 1)

    'attributes': {             # select optional attributes from the database
        'years': [2012, 2015],  # list of (not consecutive) years from 2012 to 2020, [] = ignore attribute
        'sector': 'no',
        'revenue': 'no',
        'geo_area': 'no',
    },

    # alpha parameters for hypothesis tests
    'alpha_concentration': 0.05,
    'alpha_heterogeneity': 0.01,
    'alpha_homogeneity': 0.05,

    # number of shots for dwave solver
    'shots': 10000,

    # enable / disable constraints
    'constraints': {
        'one_class': True,
        'logic': True,
        'monotonicity': True,
        'conentration': True,
        'min_thr': True,
        'max_thr': True
    },
    
    # set the mu parameters for each contraint
    'mu': {
        'one_class': 100,
        'logic': 100,
        'monotonicity': 10,
        'concentration': 20,
        'min_thr': 10,
        'max_thr': 10
    },

    # enable / disable tests
    'test': {
        'logic': True,
        'monotonicity': True,
        'conentration': True,
        'min_thr': True,
        'max_thr': True,
        'heterogeneity': True,
        'homogeneity': True
    }
}

## Constraints definitions

Sections 1, 2 and 3 of the WP5 report contain the logical and the financial constraints description for the rating scale definition problem.

To recap, we define a two-index binary variable:
$$
x_{ij} =
\begin{cases}
1 & \text{if the counterpart \(i\) is in the grade \(j\)} \\
0 & \text{otherwise}
\end{cases}
$$
that can be represented as elements of a matrix $x$ of size $n*m$, where $n$ is the number of counterparts and $m$ is the number of grades. Several constraints must be fulfilled to find the correct values of the matrix elements. These constraints are formulated as penalties of a QUBO problem. Solving this QUBO formulation allows us to classify the $n$ counterparts into the $m$ grades.

Chapter 6 report the mathematical definition of the constraints for the QUBO formulation. Here, we provide only a brief overview of the constraints:
* **Logic constraint**: given the n counterparts ordered by score, the result of the QUBO problem must be a binary staircase matrix;
* **Monotonicity constraint**: the default rate of the grades increases as the grade index increases;
* **Heterogeneity constraint**: the default values of two consecutive grades must differ significantly from each other;
* **Concentration constraint**: the counterparts are divided into grades to avoid high concentrations of the default rate;
* **Grade cardinality threshold constraints**: the number of counterparts per grade is limited from above and below.

In the following sections, we will implement a function for each of them, except for the heterogeneity one. Indeed, we considered not implementing the heterogeneity constraint in the cost function because including it would introduce too many variables, making the entire problem intractable. The main reason for this is that the heterogeneity constraint we introduced in the Section 6.3 of the WP5 report is of degree 24. Consequently, the corresponding quadratic term that enforces this requirement in the cost function introduces an exponential number of variables, which are necessary for its quadratization. As with the homogeneity test described in Chapter 4 of the WP5 report, we decided to implement only the test for the heterogeneity constraint.

We provide a brief description of the code for each constraint below, with a focus on those that required slight changes from what was described in the WP5 report.

### Logic constraint

As reported in Appendix B of the WP5 report, the contribution of the logic constraint to the cost function is:
$$
\begin{align} 
    P^{(0)}(x_{ij}) =& \mu_{01} \cdot (1-x_{11})\biggl(\sum_{j=2}^{m} x_{1j}\biggr)\\
    &+\mu_{02} \cdot (1-x_{nm})\biggl(\sum_{j=1}^{m-1} x_{nj}\biggr)\\
    &+\mu_{03} \sum_{i} \biggl(\sum_{j}x_{ij}-1\biggr)^{2}\\
    &+\mu_{04} \sum_{i,j=1}^{n-1,m-1} \Bigl[(1-x_{i+1,j}-x_{i+1,j+1})x_{ij}+x_{i+1,j}x_{i+1,j+1}\Bigr]\\
    &+\mu_{05} \sum_{i,j=1}^{n-1,m-1} \Bigl[(1-x_{ij}-x_{i,j+1})x_{i+1,j+1}+x_{ij}x_{i,j+1}\Bigr]\\
    &+\mu_{06} \sum_{i,j=1}^{n-1,m-1} x_{i,j+1}x_{i+1,j}\\
    &+\mu_{07} \sum_{i=1}^{n-1} \Bigl[(1-x_{ij}-x_{i,j+1})x_{i+1,j} + x_{ij}x_{i,j+1}\Bigr]
\end{align}
$$
where:
* $(1)$ expresses that the first counterpart must be in the first grade;
* $(2)$ expresses that the last counterpart must be in the last grade;
* $(3)$ expresses that one class must be in one and only one class;
* $(4)$, $(5)$ and $(6)$ avoid the submatrices, respectivly:

$$
\begin{align*} 
\begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}, \quad
\begin{pmatrix}
0 & 0 \\
0 & 1
\end{pmatrix}, \quad
\begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}
\end{align*};
$$

* $(7)$ avoids the following submatrix for only the first column (meaning that we penalize restarting from grade 0):
$$
\begin{align*} 
\begin{pmatrix}
0 & 0 \\
1 & 0
\end{pmatrix}
\end{align*};
$$

The following cell contains the functions that implement all these constraints.

We emphasize that penalties $1$ and $2$ are implemented in functions with descriptive names that reflect their purpose. These functions are called by `staircase_constr()`, which also implements penalties $4$, $5$, and $6$. To simplify, $\mu_{01}=\mu_{02}=\mu_{04}=\mu_{05}=\mu_{06}=\mu_{07}$, whereas $\mu_{03}$ and its corresponding penalty are kept independent to allow greater flexibility in testing. 

In [21]:
# penalty: "first counterpart in first class"
def first_counterpart_const(m, n, mu=1):
    Q = np.zeros([n*m, n*m])
    
    for jj in range(1, m):
        Q[jj][jj] += 1
        Q[0][jj] -= 0.5
        Q[jj][0] -= 0.5
    return mu*Q

# penalty: "last counterpart in the last class"
def last_counterpart_const(m, n, mu=1):
    Q = np.zeros([n*m, n*m])

    for jj in range(m-1):
        tt = (n-1)*m+jj
        Q[tt][tt] += 1
        Q[(n*m)-1][tt] -= 0.5
        Q[tt][(n*m)-1] -= 0.5
    return mu*Q

# penalty: "one class per counterpart"
def one_class_const(m, n, mu=1):
    Q = np.zeros([n*m, n*m])
    c = 0

    for ii in range(n):
        for jj in range(m):
            tt = ii*m+jj
            Q[tt][tt] += -1
        for jj in range(m-1):
            for kk in range(jj+1,m):
                tt = ii*m+jj
                rr = ii*m+kk
                Q[tt][rr] += 1
                Q[rr][tt] += 1
        c += 1
    return (mu*Q, mu*c)

# penalty: "staircase matrix"
def staircase_constr(m, n, mu=1):
    Q = first_counterpart_const(m,n) + last_counterpart_const(m,n)

    # penalize not permitted submatrix, where a submatrix is
    # [[x1, x1], [x3, x4]]
    for ii in range(n-1):

        # penalize: [[1 0],[0 0]], [[0 0],[0 1]], [[0 1],[1 0]]
        for jj in range(m-1):
            x1 = ii*m+jj
            x2 = x1+1
            x3 = (ii+1)*m+jj
            x4 = x3+1

            # add linear terms
            Q[x1][x1] += 1
            Q[x4][x4] += 1

            # add quadratic terms
            Q[x1][x2] += 0.5
            Q[x2][x1] += 0.5

            Q[x1][x3] -= 0.5
            Q[x3][x1] -= 0.5

            Q[x1][x4] -= 1
            Q[x4][x1] -= 1

            Q[x2][x3] += 0.5
            Q[x3][x2] += 0.5

            Q[x2][x4] -= 0.5
            Q[x4][x2] -= 0.5

            Q[x3][x4] += 0.5
            Q[x4][x3] += 0.5

        # penalize restarting from class 0
        x1 = ii*m
        x2 = x1+1
        x3 = (ii+1)*m

        Q[x3][x3] += 1

        Q[x1][x3] -= 0.5
        Q[x3][x1] -= 0.5

        Q[x2][x3] -= 0.5
        Q[x3][x2] -= 0.5

        Q[x1][x2] += 0.5
        Q[x2][x1] += 0.5

    return mu*Q

### Monotonicity constraint

As described in Section 6.2 of WP5, the monotonicity component in the cost function requires an exponential number of additional variables. Besides the slacks, it involves the variables necessary for quadratizing the term under consideration. To minimize the problem size, this section presents a new formulation of the monotonicity constraint. This results in a significant reduction in the number of additional variables required.

We express the DRs in terms of $x$:
$$
\begin{align} \tag{8}
\ell_{j}(x) = \frac{1}{N_{j}(x)} \sum_{i} d_{i}x_{ij} \quad \text{and} \quad \ell_{j+1}(x) = \frac{1}{N_{j+1}(x)} \sum_{i}d_{i} x_{ij+1}.
\end{align}
$$
As we know, the index $i$ is defined in $\mathbb{N}[1,n]$, i. e. the 1-based set of integers less than or equal to $n$, and every $d_i$ is a binary parameter.
The $x_{ij}$ is a binary $n \times m$ matrix. The $N_{j}(x)$ is
$$
\begin{align} \tag{9.1}
	N_{j} (x) = \sum_i x_{ij}
\end{align}
$$
for every $j=1,2,\dots,m$.
Therefore, besides $N_{j}(x)>0$ (where $j=1,2,\dots,m$), the monotonicity constraint ${\ell_{j}(x) \le \ell_{j+1}(x)}$ is
$$
\begin{align} \tag{9.2}
	\sum_{i_{1}i_{2}}d_{i_{1}i_{2}}x_{i_{1}j} x_{i_{2}j+1} \le 0
\end{align}
$$
where $d_{i_1 i_2} \in \{-1,0,+1\}$ is
$$
\begin{align} \tag{10}
	d_{i_1 i_2} = d_{i_1} - d_{i_2}.
\end{align}
$$
Such a parameter is antisymmetric: $d_{i_1 i_2} = -d_{i_2 i_1}$.
In the monotonicity constraint $j=1,2,\dots,m-1$; from now on, in this treatment we well omit that $1,2,\dots,m-1$ is the range of $j$ that we use.
Let's consider
$$
\begin{align} \tag{11}
	C^{-} = \{(i_1 i_2) \in \mathbb{N}[1,n]^2 | d_{i_1 i_2} = -1\}
\end{align}
$$
and
$$
\begin{align} \tag{12}
	C^{+} = \{(i_1 i_2) \in \mathbb{N}[1,n]^2 | d_{i_1 i_2} = 1\}
\end{align}
$$
where we avoid using commas to separate the elements of a tuple of indices.
We notice that
$$
\begin{align} \tag{13}
	\text{if} \ (i_1 i_2) \in C^{-} \ \text{then} \ (i_2 i_1) \in  C^{+}
\end{align}
$$
and
$$
\begin{align}\tag{14}
	\text{if} \ (i_1 i_2) \in C^{+} \ \text{then} \ (i_2 i_1) \in  C^{-}.
\end{align}
$$
Let us introduce
$$
\begin{align}\tag{15}
	d \equiv|\vec{d}| = \sum_{i}d_i.
\end{align}
$$

If $\vec{d}=(0 \ 1 \ 1 \ 0 \ 0)$ ($n$=5) we get $d=2$ with
$$
\begin{align}\tag{16}
	d_{i_1 i_2} =
	\left[ {\begin{array}{ccccc}
			0 & -1 & -1 & 0 & 0\\
			1 & 0 & 0 & 1 & 1\\
			1 & 0 & 0 & 1 & 1\\
			0 & -1 & -1 & 0 & 0\\
			0 & -1 & -1 & 0 & 0 
	\end{array} } \right] .
\end{align}
$$
The number of elements with $d_{i_1 i_2}=-1$ is $|C^{-}|=|C^{+}|=(n-d)d$, in this case 6.

Let's look at the $m-1$ inequalities in the form:
$$
\begin{align} \tag{17}
	\sum_{(i_1 i_2)  \in C^-} (x_{i_2 j}x_{i_1 j+1}-x_{i_1 j}x_{i_2 j+1}) \le 0 .
\end{align}
$$
These are equivalent to the system of $m-1$ equations
$$
\begin{align} \tag{18}
	\sum_{(i_1 i_2)  \in C^-} (-x_{i_2 j}x_{i_1 j+1}+x_{i_1 j}x_{i_2 j+1}) - S_{yj} = 0
\end{align}
$$
where the integer variable $S_{yj}$ is defined by
$$
\begin{align} \tag{19}
	0 \le S_{yj} \le - \min \sum_{(i_1 i_2)  \in C^-} (x_{i_2 j}x_{i_1 j+1}-x_{i_1 j}x_{i_2 j+1}) = (n-d)d .
\end{align}
$$
In this way we obtain
$$
\begin{align}\tag{20}
	\sum_{(i_1 i_2)  \in C^-} x_{i_2 j}x_{i_1 j+1}- \sum_{(i_1 i_2)  \in C^-} x_{i_1 j}x_{i_2 j+1}+ \sum_{l=0}^{N_y-1} 2^l s_{ylj} = 0
\end{align}
$$
where
$$
\begin{align}\tag{21}
	N_y = \left \lfloor 1+ \log_2 [(n-d)d] \right \rfloor.
\end{align}
$$
It is clear that the problem is well-defined only for $\vec{d}$ such that $d\in \mathbb{N}[1,n-1]$.
Notice that $s_{ylj}$ is a $N_y \times (m-1)$  binary matrix.

The resulting system is composed of $m-1$ quadratic equations.

Instead of quadratize the square of the LHS of each equation, we first linearize it and then we square it.
In order to obtain its QUBO contribution we have to linearize it.
For this reason we consider the products
$$
\begin{align}\tag{22}
	x_{i_{1}j} x_{i_{2}j+1} \ \text{for every} \ (i_1 i_2) \in C \ \text{and for every} \ j
\end{align}
$$
where
$$
\begin{align}\tag{23}
	C = C^- \cup C^+ .
\end{align}
$$
The cardinality of $C$ is $2(n-d)d$.
We set
$$
\begin{align}\tag{24}
	Q_3 = \{ (i_1 i_2 j) \in \mathbb{N}[1,n]^2 \times \mathbb{N}[1,m-1] |  (i_1 i_2) \in C \}
\end{align}
$$
Now we consider
$$
\begin{align}\tag{25}
	y_{t(i_1 i_2 j)} \equiv x_{i_{1}j} x_{i_{2}j+1} \ \text{for every} \ (i_1 i_2 j) \in Q_3
\end{align}
$$
where $t$ is a bijection defined in the set of the triples $(i_1 i_2 j)\in Q_3$ onto
the 0-based set of indices having cardinality $2(m-1)(n-d)d$. We now introduce the couples of 0-based indices
(indices flattening formulae)
$$
\begin{gather}\tag{26}
	(u_1 u_2)= ((i_1-1)m+j-1,(i_2-1)m+j-1) \ \text{for every}\  (i_1 i_2 j)\in Q_3
\end{gather}
$$
and we define
$$
\begin{align}\tag{27}
	t(u_1  u_2) = t(i_1  i_2  j)
\end{align}
$$
where $(u_1  u_2)$ and $(i_1  i_2  j)$ are linked by the indices flattening formulae. The couples $(u_1  u_2)$ are part of the set
$$
\begin{align}
	U_2 = \{(u_1 u_2)\in \mathbb{N}[0,nm-1]^2|&u_r = (i_r-1)m+j-1,\ \text{for every}\ r\in\{1,2\},\
	\nonumber\\&
	(i_1 i_2 j)\in Q_3  \} . \tag{28}
\end{align}
$$
The binary constraint $y=x_1 x_2$ is implemented through the Rosenberg’s polynomial
$$
\begin{align}
	P(x_1, x_2, y) = \mu_0 \cdot ( x_1 x_2 +3y -2x_1 y -2x_2 y )
\end{align}
$$
where $\mu_0$ is a suitable positive real number. Notice that $P(x_1, x_2, y) > 0$ if and only if $y \ne x_1 x_2$; on the contrary $P(x_1, x_2, y) = 0$ if and only if $y = x_1 x_2$. For every  $(i_1 i_2 j)\in Q_3$ we introduce the QUBO contribution
$$
\begin{align}\tag{29}
	P(x_{i_1 j}, x_{i_2 j+1}, y_{t(i_1 i_2 j)}) = \mu_0 \cdot ( x_{i_1 j} x_{i_2 j+1} +3y_{t(i_1 i_2 j)} -2x_{i_1} y_{t(i_1 i_2 j)} -2x_{i_2  j+1} y_{t(i_1 i_2 j)} )
\end{align}
$$
that is equivalent to
$$
\begin{align}\tag{30}
	P(x_{u_1}, x_{u_2+1}, y_{t(u_1 u_2)}) = \mu_0 \cdot ( x_{u_1} x_{u_2 +1} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2 +1} y_{t(u_1 u_2)} ).
\end{align}
$$
The resulting contribution to the cost function for these terms  is
$$
\begin{align}\tag{31}
	\sum_{ (u_1 u_2) \in U_2 } \mu_0 \cdot ( x_{u_1} x_{u_2+1} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2 +1} y_{t(u_1 u_2)} ) .
\end{align}
$$
Fixing a large positive real number $\mu$,
from squaring each equation and summing on $j$ we get
$$
\begin{align}
	\mu\sum_j
	\bigg[&
	\sum_{((i_1 i_2),(i_3 i_4))  \in (C^{-})^2  } y_{t(i_2 i_1 j)} y_{t(i_4 i_3 j)}
	+\sum_{((i_1 i_2),(i_3 i_4))  \in (C^{-})^2  } y_{t(i_1 i_2 j)} y_{t(i_3 i_4 j)}
	+\sum_{l_1 l_2=0}^{N_y-1} 2^{l_1+l_2} s_{yl_1 j} s_{yl_2 j}\nonumber\\&
	-2\sum_{((i_1 i_2),(i_3 i_4))  \in (C^{-})^2  }y_{t(i_2 i_1 j)} y_{t(i_3 i_4 j)}
	+2\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_2 i_1 j)}2^ls_{ylj}\nonumber\\&
	-2\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_1  i_2 j)}2^ls_{ylj} \tag{32}
	\bigg]
\end{align}
$$
the cost function term related to our problem.



Now we collect the terms:
$$
\begin{align}
	\Xi^{(1)}(x,y,s_y) =&
	\sum_{ (u_1 u_2) \in U_2}
	\mu_0\cdot ( x_{u_1} x_{u_2 +1} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2 +1} y_{t(u_1 u_2)} )
	\nonumber\\&
	+ \mu    \sum_j \bigg[ 
	\sum_{((i_1 i_2),(i_3 i_4))  \in (C^{-})^2 } y_{t(i_2 i_1 j)} y_{t(i_4 i_3 j)}
	+\sum_{((i_1 i_2),(i_3 i_4))  \in (C^{-})^2 } y_{t(i_1 i_2 j)} y_{t(i_3 i_4 j)}\nonumber\\&
	+\sum_{l_1 l_2=0}^{N_y-1} 2^{l_1+l_2}s_{yl_1 j}s_{y l_2 j}
	-2\sum_{((i_1 i_2),(i_3 i_4))  \in (C^{-})^2  }y_{t(i_2 i_1 j)} y_{t(i_3 i_4 j)} \nonumber\\&
	+2\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_2 i_1 j)}2^ls_{y lj}  \tag{33}
	-2\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_1  i_2 j)}2^ls_{ylj}
	\bigg] .
\end{align}
$$
Now let's look at the summations involving $(C^{-})^2$. By introducing
$$
\begin{align}
	U_4 = \{ (u_1 u_2 u_3 u_4) \in \mathbb{N}[0,nm-1]^4|&u_r = (i_r-1)m+j-1, \text{for every}\nonumber\\&r\in\{1,2,3,4\}, (i_1,i_2)\in C^-, (i_3,i_4)\in C^- \} \tag{34}
\end{align}
$$
we get
$$
\begin{align}
	\Xi^{(1)}(x,y,s_y) =&
	\sum_{ (u_1,u_2) \in U_2}
	\mu_0 \cdot ( x_{u_1} x_{u_2 +1} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2 +1} y_{t(u_1 u_2)} )
	\nonumber\\&
	+ \mu \cdot \bigg[
	\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_2 u_1)} y_{t(u_4 u_3)}
	+\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_1 u_2)} y_{t(u_3 u_4)}\nonumber\\&
	+\sum_j\sum_{l_1 l_2=0}^{N_y-1} 2^{l_1+l_2}s_{yl_1 j}s_{y l_2 j}
	-2\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_2 u_1)} y_{t(u_3 u_4)}\nonumber\\&
	+2\sum_j\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_2 i_1 j)}2^ls_{y lj} \tag{35}
	-2\sum_j\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_1   i_2 j)}2^ls_{ylj}
	\bigg] .
\end{align}
$$
Since $s_{ylj}$ is a $N_y \times (m-1)$ binary matrix, for the quadratic summation in $s_{ylj}$ we define
$$
\begin{align}
	V_2 =\{ (v_1   v_2) \in \mathbb{N}[0,N_y \cdot(m-1)-1] ^2|&v_r = l_r \cdot (m-1)+j-1 \ \text{for every}\
	l_r \in \mathbb{N}[0, N_y -1]   , \nonumber\\& r\in\{1,2\}    \}, \tag{36}
\end{align}
$$
and get
$$
\begin{align}
	\Xi^{(1)}(x,y,s_y) =&
	\sum_{ (u_1,u_2) \in U_2}
	\mu_0 \cdot ( x_{u_1} x_{u_2 +1} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2 +1} y_{t(u_1 u_2)} )
	\nonumber\\&
	+ \mu \cdot \bigg[
	\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_2 u_1)} y_{t(u_4 u_3)}
	+\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_1 u_2)} y_{t(u_3 u_4)}\nonumber\\&
	+\sum_{(v_1 v_2) \in V_2} 2^{l(v_1)+l(v_2)}s_{yv_1}s_{y v_2}
	-2\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_2 u_1)} y_{t(u_3 u_4)}\nonumber\\&
	+2\sum_j\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_2 i_1 j)}2^ls_{y lj}
	-2\sum_j\sum_{(i_1 i_2)  \in C^-} \sum_{l=0}^{N_y-1}  y_{t(i_1  i_2 j)}2^ls_{ylj} \tag{37}
	\bigg] .
\end{align}
$$
where
$$
\begin{align}\tag{38}
	l(v) = \left \lfloor v/(m-1)\right\rfloor.
\end{align}
$$
Now we treat the remaining two summations; by introducing
$$
\begin{align}
	H = \{(u_1 u_2 v)  \in  \mathbb{N}[0,nm- 1]^2 \times \mathbb{N}[0,N_y \cdot (m-1)-1]
	|& u_r = (i_r-1)m+j-1,\nonumber\\&v=l\cdot (m-1)+j-1 \nonumber\\& \text{for every}\ r \in \{1,2\}, \ (i_1,i_2)\in C^- ,
	\nonumber\\&
	l \in \mathbb{N}[0, N_y -1] \tag{39}
	\}
\end{align}
$$
we get
$$
\begin{align} 
	\Xi^{(1)}(x,y,s_y) =&
	\sum_{ (u_1 u_2) \in U_2}
	\mu_0 \cdot ( x_{u_1} x_{u_2 +1} +3y_{t(u_1 u_2)} -2x_{u_1} y_{t(u_1 u_2)} -2x_{u_2 +1 } y_{t(u_1 u_2)} )
	\nonumber\\&
	+ \mu \cdot \bigg[
	\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_2 u_1)} y_{t(u_4 u_3)}
	+\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_1 u_2)} y_{t(u_3 u_4)}\nonumber\\&
	+\sum_{(v_1 v_2) \in V_2} 2^{l(v_1)+l(v_2)}s_{yv_1}s_{y v_2}
	-2\sum_{(u_1 u_2 u_3 u_4)  \in U_4} y_{t(u_2 u_1)} y_{t(u_3 u_4)}\nonumber\\&
	+2\sum_{(u_1 u_2 v) \in H }  y_{t(u_2 u_1)} 2^{l(v)}s_{y v}
	-2\sum_{(u_1 u_2 v) \in H }  y_{t(u_1 u_2)} 2^{l(v)}s_{y v} \tag{40}
	\bigg] .
\end{align}
$$
We obtained
$$
\begin{align}
	\Xi^{(1)}(x,y,s_y) =&
	\sum_{ (u_1 u_2) \in U_2}
	(\mu_0x_{u_1} x_{u_2 +1 } +3\mu_0y_{t(u_1 u_2)} -2\mu_0x_{u_1} y_{t(u_1 u_2)} -2\mu_0x_{u_2 +1 } y_{t(u_1 u_2)})  
	\nonumber\\&
	+  
	\sum_{(u_1 u_2 u_3 u_4)  \in U_4} \mu y_{t(u_2 u_1)} y_{t(u_4 u_3)}
	+\sum_{(u_1 u_2 u_3 u_4)  \in U_4} \mu y_{t(u_1 u_2)} y_{t(u_3 u_4)}\nonumber\\&
	+\sum_{(v_1 v_2) \in V_2} 2^{l(v_1)+l(v_2)}\mu s_{yv_1}s_{y v_2}
	+\sum_{(u_1 u_2 u_3 u_4)  \in U_4} (-2)\mu y_{t(u_2 u_1)} y_{t(u_3 u_4)}\nonumber\\&
	+\sum_{(u_1 u_2 v) \in H }2^{l(v)+1}\mu y_{t(u_2 u_1)}s_{y v}
	+\sum_{(u_1 u_2 v) \in H }(-1) 2^{l(v)+1} \mu y_{t(u_1 u_2)}s_{y v}  \tag{41}
\end{align}
$$
where $t$ is a bijection defined in
$$
\begin{align}\tag{42}
	U_2 = \{(u_1 u_2)\in \mathbb{N}[0,nm - 1]^2|u_r = (i_r-1)m+j-1,\ \text{for every}\ r\in\{1,2\},\ (i_1 i_2 j)\in Q_3\}
\end{align}
$$
onto the 0-based indices set having cardinality $2(m-1)(n-d)d$,
$$
\begin{align}\tag{43}
	U_4 = \{ (u_1 u_2 u_3 u_4) \in \mathbb{N}[0,nm -1]^4|u_r = (i_r-1)m+j-1 \ \text{for every} \ r\in\{1,2,3,4\}, (i_1,i_2)\in C^-, (i_3,i_4)\in C^-  \},
\end{align}
$$
$$
\begin{align}\tag{44}
V_2 =\{ (v_1   v_2) \in \mathbb{N}[0,N_y \cdot(m-1)-1] ^2|v_r = l_r \cdot (m-1)+j-1 \ \text{for every} \
	l_r \in \mathbb{N}[0, N_y -1]   ,  r\in\{1,2\}    \},
\end{align}
$$
$$
\begin{align}\tag{45}
H = \{(u_1 u_2 v)  \in  \mathbb{N}[0,nm  -1]^2 \times \mathbb{N}[0,N_y \cdot (m-1)-1]
|  u_r = (i_r-1)m+j-1,v=l\cdot (m-1)+j-1 \ \text{for every}  \ r \in \{1,2\},  (i_1,i_2)\in C^-  ,
l  \in \mathbb{N}[0, N_y -1]
\},
\end{align}
$$
$$
\begin{align}\tag{46}
	l(v) = \left \lfloor v/(m-1) \right\rfloor,
\end{align}
$$
$$
\begin{align}\tag{47}
C^{\pm} = \{(i_1 i_2) \in \mathbb{N}[1,n]^2 | d_{i_1 i_2} = \pm 1\}   ,
\end{align}
$$
$$
\begin{align}\tag{48}
Q_3 = \{ (i_1 i_2 j) \in \mathbb{N}[1,n]^2 \times \mathbb{N}[1,m-1] |  (i_1 i_2) \in C \}  ,
\end{align}
$$
$$
\begin{align}\tag{49}
N_y = \left \lfloor 1+ \log_2 [(n-d)d] \right \rfloor.
\end{align}
$$

The binary vectors involved are:
$$
\begin{align}\tag{50}
	x \in \mathbb{B}^{nm},\quad
	y \in \mathbb{B}^{2(m-1)(n-d)d} ,\quad
	s_y \in \mathbb{B}  {}^{\wedge} \{ (m-1) \left \lfloor 1+ \log_2 [(n-d)d] \right \rfloor \}  .
\end{align}
$$

For $n=5$, $m=3$, $d=2$
$$
\begin{align}\tag{51}
	(n-d)d = 6
\end{align}
$$
then the number of binary variables involved is:
$$
\begin{align}\tag{52}
	nm + 2(m-1)(n-d)d+(m-1) \left \lfloor 1+ \log_2 [(n-d)d] \right \rfloor &= 15 + 24 + 6
	=45.
\end{align}
$$

In [ ]:
def monotonicity_constr(m, n, default, offset, mu=1):

    def l_func(v):
	    return math.floor(v/(m-1))

    # check if the default values are not equal
    if np.all(default==0) or np.all(default==1):
        print("Error in monotonicity function call. Default values are all equal")
        sys.exit(0)

    num_of_default = sum(default)
    param = (n-num_of_default)*num_of_default
    Ny = math.floor(1+math.log2(param))
    dim_y = 2*(m-1)*param
    offset_sy = offset + dim_y
    dim_sy = (m-1)*Ny

    dim = offset + dim_y + dim_sy
    Q = np.zeros([dim, dim])

    C_set_minus = []
    C_set = []
    for i1 in range(n):
        for i2 in range(n):
            if default[i1]-default[i2] == -1:
                C_set_minus.append([i1+1,i2+1])
            if default[i1]-default[i2] != 0:
                C_set.append([i1+1,i2+1])

    u2 = []
    for j in range(m-1):
        for item_c_set in C_set:
            u2_1 = (item_c_set[0] -1)*m + (j+1) -1
            u2_2 = (item_c_set[1] -1)*m + (j+1) -1
            u2.append([u2_1 , u2_2])

    u4 = []
    for j in range(m - 1):
        for item_c_set_minus_1 in C_set_minus:
            u_1 = (item_c_set_minus_1[0] - 1) * m + (j + 1) - 1
            u_2 = (item_c_set_minus_1[1] - 1) * m + (j + 1) - 1
            for item_c_set_minus_2 in C_set_minus:
                u_3 = (item_c_set_minus_2[0] - 1) * m + (j + 1) - 1
                u_4 = (item_c_set_minus_2[1] - 1) * m + (j + 1) - 1
                u4.append([u_1, u_2, u_3, u_4])
    
    l2j1 = []
    for l1 in range(Ny):
        for l2 in range(Ny):
            for j in range(m-1):
                l2j1.append([l1,l2,j+1])
    v2 = []
    for l2j1_item in l2j1:
        v_1 = l2j1_item[0]*(m-1) + l2j1_item[2] -1
        v_2 = l2j1_item[1]*(m-1) + l2j1_item[2] -1
        v2.append([v_1,v_2])

    h = []
    for j in range(m-1):
        for c_min_item in C_set_minus:
            u_1=(c_min_item[0]-1)*m + (j+1) -1
            u_2=(c_min_item[1]-1)*m + (j+1) -1
            for l in range(Ny):
                v = l*(m-1) + (j+1) -1
                h.append([u_1,u_2,v])

    for u2_item in u2:
        u_1 = u2_item[0]; u_2 = u2_item[1]
        if u_1==u_2+1:
            Q[u_1,u_2+1] += mu
        else:
            Q[u_1,u_2+1] += mu*0.5
            Q[u_2+1,u_1] += mu*0.5

    for u2_item in u2:
        u_1 = u2_item[0]; u_2 = u2_item[1]
        t=u2.index([u_1,u_2])
        Q[ offset + t, offset + t ] += mu*3

    for u2_item in u2:
        u_1 = u2_item[0]; u_2 = u2_item[1]
        t=u2.index([u_1,u_2])
        Q[u_1,offset + t] += mu*(-2)*0.5
        Q[offset + t,u_1] += mu*(-2)*0.5

    for u2_item in u2:
        u_1 = u2_item[0]; u_2 = u2_item[1]
        t=u2.index([u_1,u_2])
        Q[u_2+1,offset + t] += mu*(-2)*0.5
        Q[offset + t,u_2+1] += mu*(-2)*0.5

    # first summation
    for u4_item in u4:
        u_1 = u4_item[0]; u_2 = u4_item[1]
        u_3 = u4_item[2]; u_4 = u4_item[3]
        t21 = u2.index([u_2,u_1])
        t43 = u2.index([u_4,u_3])
        if t21==t43:
            Q[ offset + t21 , offset + t43 ] += mu
        else:
            Q[ offset + t21 , offset + t43 ] += mu*0.5
            Q[ offset + t43 , offset + t21 ] += mu*0.5
    # second summation
    for u4_item in u4:
        u_1 = u4_item[0]; u_2 = u4_item[1]
        u_3 = u4_item[2]; u_4 = u4_item[3]
        t12 = u2.index([u_1,u_2])
        t34 = u2.index([u_3,u_4])
        if t12==t34:
            Q[ offset + t12 , offset + t34 ] += mu
        else:
            Q[ offset + t12 , offset + t34 ] += mu*0.5
            Q[ offset + t34 , offset + t12 ] += mu*0.5
            
    # first summation
    for v2_item in v2:	
        v_1 = v2_item[0]; v_2 = v2_item[1]
        if v_1==v_2:
            Q[ offset_sy + v_1 , offset_sy + v_2 ] += math.pow( 2, l_func(v_1) + l_func(v_2) )*mu
        else:
            Q[ offset_sy + v_1 , offset_sy + v_2 ] += math.pow( 2, l_func(v_1) + l_func(v_2) )*mu*0.5
            Q[ offset_sy + v_2 , offset_sy + v_1 ] += math.pow( 2, l_func(v_1) + l_func(v_2) )*mu*0.5
    # second summation
    for u4_item in u4:
        u_1 = u4_item[0]; u_2 = u4_item[1]
        u_3 = u4_item[2]; u_4 = u4_item[3]
        t21 = u2.index([u_2,u_1])
        t34 = u2.index([u_3,u_4])
        if t21==t34:
            Q[ offset + t21 , offset + t34 ] += mu*(-2)
        else:
            Q[ offset + t21 , offset + t34 ] += mu*(-2)*0.5
            Q[ offset + t34 , offset + t21 ] += mu*(-2)*0.5

    # first summation
    for h_item in h:
        u_1 = h_item[0]; u_2 = h_item[1]; v = h_item[2]
        t21 = u2.index([u_2,u_1])
        Q[ offset + t21 , offset_sy + v ] += math.pow( 2, l_func(v) + 1 )*mu*0.5
        Q[ offset_sy + v , offset + t21 ] += math.pow( 2, l_func(v) + 1 )*mu*0.5
    # second summation
    for h_item in h:
        u_1 = h_item[0]; u_2 = h_item[1]; v = h_item[2]
        t12 = u2.index([u_1,u_2])
        Q[ offset + t12 , offset_sy + v ] += (-1)*math.pow( 2, l_func(v) + 1 )*mu*0.5
        Q[ offset_sy + v , offset + t12 ] += (-1)*math.pow( 2, l_func(v) + 1 )*mu*0.5

    return Q

### Concentration constraint

We have introduced certain modifications for this constraint with respect to what we defined in the WP5 report. The cost function component presented in Section 6.4 scales as $O(n^4m^2)$ in terms of additional variables. Consequently, including this constraint in problems of realistic size would result in exponential growth of the number of variables, making the overall problem computationally intractable. To mitigate this issue and reduce the dimensionality of the variable space, we propose the following approximation of this constraint:
$$
\min_ x H_{adj}(x)
$$
where
$$
\begin{align}\tag{53}
H_{adj}(x) = \frac{\sum_{i_1 i_2 j} x_{i_1 j} x_{i_2 j} - \frac{1}{m}}{1-\frac{1}{m}}.
\end{align}
$$
This minimum problem has no additional binary variables.

In [ ]:
def concentration_constr(m, n, mu=1):
    Q = np.zeros([n*m, n*m])

    # penalty: "concentration"
    c = 1/(1-m)
    gamma = m/(m-1)
    
    for i1 in range(n):
        for i2 in range(n):
            for j in range(m):
                u1 = (i1)*m+j
                u2 = (i2)*m+j
                if u1==u2:
                    Q[u1][u2] += gamma
                else:
                    Q[u1][u2] += gamma/2

    return (mu*Q, mu*c)

### Grade cardinality threshold constraint

Following Appendix H of the WP5 report, the contribution of the grade cardinality threshold constraints to the cost function is as follows:

$$
\begin{align}
    P^{(4)} _S (\vec{x},\vec{s}_{41},\vec{s}_{42})
    =&
    \mu_{41S} \cdot \biggl( \left\lceil\frac{n}{100}\right\rceil ^2 m +
    \sum_{u_1 u_2} x_{u_1} x_{u_2} + \sum_{j, l_1, l_2 =0}^{N_{S}^{(41)}-1} 2^{l_1} 2^{l_2} s_{l_1 j}^{(41)} s_{l_2 j}^{(41)}
    -2 \left\lceil\frac{n}{100}\right\rceil \sum_{u} x_{u}+ \nonumber\\
    &+ 2 \left\lceil\frac{n}{100}\right\rceil
    \sum_{j, l=0}^{N_{S}^{(41)}-1} 2^l s_{lj}^{(41)}-2\sum_{u} x_{u} \sum_{j, l=0}^{N_{S}^{(41)}-1} 2^l s_{lj}^{(41)} \biggr)  \tag{54} \\
    &+\mu_{42S}  \cdot \biggl( \left\lfloor\frac{15n}{100}\right\rfloor ^2 m + \sum_{u_1 u_2}x_{u_1}x_{u_2} + \sum_{j, l_1, l_2 =0}^{N_{S}^{(42)}-1} 2^{l_1} 2^{l_2} s_{l_1 j}^{(42)} s_{l_2 j}^{(42)}
    - 2 \left\lfloor\frac{15n}{100}\right\rfloor \sum_{u} x_{u} \nonumber\\
    &- 2 \left\lfloor\frac{15n}{100}\right\rfloor \sum_{j, l=0}^{N_{S}^{(42)}-1} 2^l s_{lj}^{(42)}
    +2 \sum_{u} x_{u} \sum_{j, l=0}^{N_{S}^{(42)}-1} 2^l s_{lj}^{(42)} \biggr). \tag{55} 
\end{align}
$$

The `threshold_constr()` function below implements both the lower and the upper threshold constraints, switching between the two depending on the `minmax` parameter.

It is important to note that the first step of the function is to compute the thresholds:
* the lower threshold is  1% of the counterparts, or 1 if there are fewer than 100 counterparts.
* the upper threshold is 15% of the counterparts, unless there are fewer than seven grades or 15% is less than zero, in which case the upper threshold is set to $n-grades+1$. In these situations, there are no integer numbers such that the constraint is fulfilled.


In [1]:
def compute_lower_thrs(n):
    return math.floor(n*0.01) if math.floor(n*0.01) != 0 else 1

def compute_upper_thrs(n, grades):
    return math.floor(n*0.15) if grades > 7 and math.floor(n*0.15) != 0 else (n-grades+1)
    
def threshold_constr(m, n, offset, minmax, mu=1):

    # compute the thresholds
    if minmax == 'min':
        thr = compute_lower_thrs(n)
        slack_vars = math.floor(1+math.log2(n-thr))
    elif minmax == 'max':
        thr = compute_upper_thrs(n, m)
        slack_vars = math.floor(1+math.log2(thr))
    else:
        print("Error in threshold function call")
        sys.exit(1)

    # initialize Q and c
    dim = offset+slack_vars*m
    Q = np.zeros([dim, dim])
    c = m * thr * thr

    # quadratic term (in the variable x)
    for i1 in range(n):
        for i2 in range(n):
            for j in range(m):
                u2 = [i1*m+j, i2*m+j]
                Q[u2[0]][u2[1]] += 0.5
                Q[u2[1]][u2[0]] += 0.5

    # quadratic term (in the slack variable)
    for l1 in range(slack_vars):
        for l2 in range(slack_vars):
            for j in range(m):
                v2 = [l1*m+j, l2*m+j]
                tmp = math.pow(2,math.floor((v2[0]+1)/m)+math.floor((v2[1]+1)/m))
                Q[offset+v2[0]][offset+v2[1]] += 0.5*tmp
                Q[offset+v2[1]][offset+v2[0]] += 0.5*tmp

    # linear term (in the x variable)
    for i in range(n):
        for j in range(m):
            u = i*m+j
            Q[u,u] -= 2*thr

    # linear term (in the slack variable)
    index = 0
    for l in range(slack_vars):
        for j in range(m):
            Q[offset+index][offset+index] += thr*math.pow(2,1+math.floor((l*m+j+1)/m))
            index += 1

    # quadratic term (in the variables x and s)
    for i in range(n):
        for l in range(slack_vars):
            for j in range(m):
                w2 = [i*m+j, l*m+j]
                tmp = math.pow(2,1+math.floor((w2[1]+1)/m))
                Q[w2[0]][offset+w2[1]] -= -0.5*tmp
                Q[offset+w2[1]][w2[0]] -= -0.5*tmp

    return (mu*Q, mu*c)

## Test the constraints

In the file `check_constraint.py` we implemented the tests for all the constraints described in the report. Given a matrix and the appropriate hyperparameters, one function per constraint tests if that matrix fulfill that specific requirement.

All of the functions are properly commented, so we refer the reader directly to the code for further details.

In [25]:
from src.check_constraints import *

## The solvers of the QUBO formulation

In order to properly classify the counterparts in the grades, we must solve the QUBO formulation. This means we must find the $x$ vector that minimizes the *cost function*:
$$
C(x) = x^t Q x + c .
$$
To accomplish this, we implemented several functions that explore different approaches.

### Exact solvers

First of all, the `brute_force_solver` function tests all the possible solutions of the system by comparing the value of the cost function for each of them. Since the $x$ vector has dimensions of $m*n$, we must test $2^{m*n}$ possible solutions. This means that this function is useful only for testing very small instances of our problem.

In [26]:
def brute_force_solver(Q, c, dim):

    # compute C(Y) = (Y^T)QY + (G^T)Y + c for every Y
    Ylist = list(itertools.product([0, 1], repeat=dim))
    Cmin = float('inf')

    for ii in range(len(Ylist)):
        Y = np.array(Ylist[ii])
        Cy=(Y.dot(Q).dot(Y.transpose()))+c
        if ( Cy < Cmin ):
            Cmin = Cy
            Ymin = Y.copy()
    
    return (np.array(Ymin), Cmin)

The second approach uses the functions provided by the D-Wave Ocean libraries. Specifically, the `exact_solver` function relies on the [dimod library](https://docs.dwavequantum.com/en/latest/ocean/api_ref_dimod/), where the `ExactSolver` method is implemented. Once a QUBO problem has been properly formatted, this method calculates the energy of all possible samples. However, it is very slow and it is therefore only suggested for testing small problem instances. Attempting to use it with larger instances results in an error related to insufficient memory to complete execution.

The input format required by the library is a Python dictionary, in which the keys represent the indices of the elements of the Q matrix, while the values correspond to the matrix entries themselves. This representation is known as a *binary quadratic model* (BQM).

Since the next solver requires an object of type `BinaryQuadraticModel`, we developed a function that generates the corresponding *dimod* object given the $Q$ matrix and the constant $c$ of the QUBO problem.

In [27]:
def from_matrix_to_bqm(matrix, c):
    
    Q_dict = {(i, j): matrix[i, j] for i in range(matrix.shape[0]) for j in range(matrix.shape[1])}# if matrix[i, j] != 0}
    bqm = dimod.BinaryQuadraticModel.from_qubo(Q_dict, c)

    return bqm

def exact_solver(bqm):
    
    sampler = dimod.ExactSolver()
    sampleset = sampler.sample(bqm)

    return sampleset

### Annealing simulator

Until now, we have presented exact approaches with an exponential complexity. In the next cell, we will use another method from the Ocean D-Wave libraries to exploit the features of quantum computing: the `SimulatedAnnealingProblemSampler`.

As its name suggests, this method simulates the behavior of a quantum annealer to find the solution to a BQM. To obtain a solution, the following parameters are required: an initial state and the number of *shots*, i.e., the number of iterations to attempt before returning the best solution found up to that point.
Therefore, the solution returned by this function may not fully satisfy all the constraints of the problem. However, the advantage of using this method is that its computational complexity is linear with respect to the problem size.

It is important to note that D-Wave provides an analogous function that can be run directly on its quantum annealers. Due to limited access to such hardware, it is preferable to first validate the code using classical quantum simulators. For the purposes of this notebook, we therefore used the `SimulatedAnnealingProblemSampler` function to verify that the implemented algorithm actually produces acceptable results.

In [28]:
def annealer_solver(dim, bqm, shots):

    # define the initial state (all elements = 0 or random elements)
    state = hybrid.core.State.from_sample({i: 0 for i in range(dim)}, bqm)
    # state = hybrid.core.State.from_sample({i: np.random.randint(0, 2) for i in range(dim)}, bqm)

    sampler = hybrid.samplers.SimulatedAnnealingProblemSampler(num_sweeps=shots)
    result_state = sampler.run(state).result()
 
    return result_state

## Definition and resolution of a problem instance

Now that we have defined all the building blocks of our problem, we can create an instance of the problem and solve it using one or more of the methods introduced earlier. Then, we can verify that the solution meets all the problem's constraints.

The following code instantiates a dataset using the hyperparameters specified at the beginning of the notebook. Specifically, we can choose to generate a random dataset or read the counterparts from the one provided by Intesa Sanpaolo.

It is important to note that in both cases, the dataset is ordered by score: the counterpart with the lowest score is first and the counterpart with the highest score is last. This is a mandatory requirement for the algorithm.

After selecting the dataset, we build the QUBO formulation of that instance. We computed the $Q$ matrix and the constant $c$ by calling all the constraint functions that were defined before. For testing purposes, we implemented the choice of which constraints to implement using the appropriate hyperparameters in the `constraint` dictionary of the `config`.

Finally, we convert the NumPy matrix $Q$ into the corresponding `BinaryQuadraticModel` object.




In [ ]:
from src.select_data import *

# generate a random dataset or select data from the dataset
dataset = generate_data(config) if config['random_data'] == 'yes' else load_data(config)
n = len(dataset)
m = config['grades']
default = dataset['default'].to_numpy().reshape(n,1)

# generate the appropriate Q matrix
start_time = time.perf_counter_ns()
Q = np.zeros([m*n, m*n])
c = 0
if config['constraints']['one_class'] == True:
    (Q_one_class,c_one_class) = one_class_const(m,n,config['mu']['one_class'])
    Q = Q + Q_one_class
    c = c + c_one_class
if config['constraints']['logic'] == True:
    Q = Q + staircase_constr(m,n,config['mu']['logic'])
if config['constraints']['conentration'] == True:
    (Q_conc,c_conc) = concentration_constr(m, n, config['mu']['concentration'])
    Q = Q + Q_conc
    c = c + c_conc
if config['constraints']['monotonicity'] == True:
    Q_monoton = monotonicity_constr(m, n, default.T.squeeze(), Q.shape[0], config['mu']['monotonicity'], config['mu']['monotonicity'], config['mu']['monotonicity'])
    pad = Q_monoton.shape[0] - Q.shape[0]
    Q = np.pad(Q, pad_width=((0,pad), (0, pad)), mode='constant', constant_values=0) + Q_monoton
if config['constraints']['min_thr'] == True:
    (Q_min_thr, c_min_thr) = threshold_constr(m, n, Q.shape[0], 'min', config['mu']['min_thr'])
    pad = Q_min_thr.shape[0] - Q.shape[0]
    Q = np.pad(Q, pad_width=((0,pad), (0, pad)), mode='constant', constant_values=0) + Q_min_thr
    c = c + c_min_thr
if config['constraints']['max_thr'] == True:
    (Q_max_thr, c_max_thr) = threshold_constr(m, n, Q.shape[0], 'max', config['mu']['max_thr'])
    pad = Q_max_thr.shape[0] - Q.shape[0]
    Q = np.pad(Q, pad_width=((0,pad), (0, pad)), mode='constant', constant_values=0) + Q_max_thr
    c = c + c_max_thr
end_time = time.perf_counter_ns()

# generate the BMQ
bqm = from_matrix_to_bqm(Q, c)

print(f"Matrix size:{Q.shape}")
print(f"Time of generation: {(end_time - start_time)/10e9} s")

Matrix size:(24, 24)
Time of generation: 9.00106e-05 s


The following cells attempt to solve the QUBO problem using the aforementioned solvers.
* with the brute force solver
* with the exact solver by D-Wave
* with the annealing solver by D-Wave

These tests were conducted on a very small instance for demonstrative purposes only. For more accurate and extensive tests, refer to the documents that will be provided in the next milestone.

In [31]:
# Solving with brute force
start_time = time.perf_counter_ns()
(result_bf, cost) = brute_force_solver(Q,c,Q.shape[0])
end_time = time.perf_counter_ns()
if config['constraints']['min_thr'] == True:
    result_bf = result_bf[:m*n]
print(f"\nBrute Force result:\n{result_bf.reshape(n,m)}")
print(f"Time of brute force solution: {(end_time - start_time)/10e9} s\n")


Brute Force result:
[[1 0]
 [1 0]
 [0 1]
 [0 1]]
Time of brute force solution: 4.3073394784 s



In [32]:
# Solving exactly with dwave
start_time = time.perf_counter_ns()
e_result = exact_solver(bqm)
df_result = e_result.lowest().to_pandas_dataframe()
end_time = time.perf_counter_ns()
elapsed_time_ns = end_time - start_time
# Print all the solutions
result_exact_solver = df_result.iloc[:, :m*n].to_numpy()
# print(f"All exact solutions:\n{df_result}")
print(f"Exact solutions with dwave: {int(result_exact_solver.size/(m*n))}")
for sol in result_exact_solver[:]:
    print(f"solution:\n{sol.reshape(n, m)}")
print(f"Time of all exact solutions: {elapsed_time_ns/10e9} s")
# print(f"First solution:\n{result_exact_solver[0].reshape(n, m)}")

Exact solutions with dwave: 1
solution:
[[1 0]
 [1 0]
 [0 1]
 [0 1]]
Time of all exact solutions: 1.4539689941 s


In [36]:
# Solving with annealing 
start_time = time.perf_counter_ns()
result = annealer_solver(Q.shape[0], bqm, config['shots'])
end_time = time.perf_counter_ns()
result_ann = np.array([int(x) for x in result.samples.first.sample.values()])[:m*n]
annealing_matrix = result_ann.reshape(n, m)
print(f"\nAnnealing result:\n{annealing_matrix}")    
print(f"\nTime of annealing solution: {(end_time - start_time)/10e9} s\n")


Annealing result:
[[1 0]
 [1 0]
 [0 1]
 [0 1]]

Time of annealing solution: 0.0008014533 s



## Result validation

Finally, we test whether the annealing solution fulfills all the constraints of the rating scale definition problem.

Note that, due to the small size of the instance, the heterogeneity and homogeneity constraints could not be satisfied because a large number of counterparts were required to perform the t-test and z-test. Additionally, due to the size of the problem, the monotonicity constraint may not have been fulfilled because it is strictly related to the generated dataset, particularly the counterparts' defaults.

In [37]:
print("Result validation:")
verbose = True
is_valid = test_one_solution(annealing_matrix, config, n, m, default, compute_upper_thrs(n,m), compute_lower_thrs(n), verbose)

Result validation:
Checking the constraints...
	✓ Logic constraint checked
	✓ Monotonicity constraint checked
	✓ Concentration constraint checked
	✓ Upper threshold limit constraint checked
	✓ Lower threshold limit constraint checked
	x Error: heterogeneous constraint not respected
		 Grades 0 and 1 are not heterogeneous
	x Error in homogeneity constraint: at least one grade has less than 2 elements
Test time: 7.63109e-05 s
